In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
import requests
import os
import csv
from langchain_teddynote.prompts import load_prompt
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "up_Mw8PEw6k3ehZqrZbQ9WfUcCPIysnn"

url = "https://api.upstage.ai/v1/document-ai/ocr"
headers = {"Authorization": f"Bearer {api_key}"}

image_folder = "image"
image_files = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png"))]

prompt = load_prompt("prompt/crops.yaml", encoding="utf-8")

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = prompt | llm | StrOutputParser()

In [3]:
for filename in image_files:
    # Open and send each image file to the OCR API
    with open(os.path.join(image_folder, filename), "rb") as file:
        files = {"document": file}
        response = requests.post(url, headers=headers, files=files)
        response_data = response.json()

    # Extract text from response
    pages = response_data.get("pages", [])
    if not pages:
        print(f"OCR 결과에 'pages' 데이터가 없습니다. 파일: {filename}")
        continue  # pages가 비어있으면 해당 파일 건너뛰기

    texts = [page.get("text", "") for page in pages]
    if not texts or not texts[0]:
        print(f"OCR 결과에서 텍스트를 추출하지 못했습니다. 파일: {filename}")
        continue  # 텍스트가 비어있을 경우 건너뛰기
    data = texts[0].replace(",", "")

    # Process the result with the language model
    result = chain.invoke(
        {"context": data, "question": "context를 csv 형식으로 변환해줘"}
    )

    # Save the result to a CSV file
    csv_filename = f"{os.path.splitext(filename)[0]}.csv"
    csv_data = result.split("\n")

    filtered_data = []
    record = False
    for line in csv_data:
        if '"""' in line or "```" in line:
            continue
        if "상품명" in line:
            record = True
        if record:
            filtered_data.append(line)

    with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as file:
        writer = csv.writer(file)
        for row in filtered_data:
            writer.writerow(row.split(","))

    print(f"CSV 파일이 {csv_filename}로 저장되었습니다.")

CSV 파일이 KakaoTalk_20250226_111541090.csv로 저장되었습니다.
CSV 파일이 KakaoTalk_20250226_111541090_01.csv로 저장되었습니다.
CSV 파일이 KakaoTalk_20250226_111541090_02.csv로 저장되었습니다.
CSV 파일이 KakaoTalk_20250226_111541090_03.csv로 저장되었습니다.
CSV 파일이 KakaoTalk_20250226_111541090_04.csv로 저장되었습니다.


KeyboardInterrupt: 